In [0]:
import pandas as pd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data] | 
[nltk_data] | Downloading package abc to /root/nltk_data...
[nltk_data] | Package abc is already up-to-date!
[nltk_data] | Downloading package alpino to /root/nltk_data...
[nltk_data] | Package alpino is already up-to-date!
[nltk_data] | Downloading package biocreative_ppi to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package biocreative_ppi is already up-to-date!
[nltk_data] | Downloading package brown to /root/nltk_data...
[nltk_data] | Package brown is already up-to-date!
[nltk_data] | Downloading package brown_tei to /root/nltk_data...
[nltk_data] | Package brown_tei is already up-to-date!
[nltk_data] | Downloading package cess_cat to /root/nltk_data...
[nltk_data] | Package cess_cat is already up-to-date!
[nltk_data] | Downloading package cess_esp to /root/nltk_data...
[nltk_data] | Package cess_esp is already up-to-date!
[nltk_data] | Downloading package chat80 to /root/nltk_data...
[nltk_data] | Package chat80 is already up-to-date!
[nltk_data] | Downloading package city_database to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package city_database is already up-to-date!
[nltk_data] | Downloading package cmudict to /root/nltk_data...
[nltk_data] | Package cmudict is already up-to-date!
[nltk_data] | Downloading package comparative_sentences to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package comparative_sentences is already up-to-
[nltk_data] | date!
[nltk_data] | Downloading package comtrans to /root/nltk_data...
[nltk_data] | Package comtrans is already up-to-date!
[nltk_data] | Downloading package conll2000 to /root/nltk_data...
[nltk_data] | Package conll2000 is already up-to-date!
[nltk_data] | Downloading package conll2002 to /root/nltk_data...
[nltk_data] | Package conll2002 is already up-to-date!
[nltk_data] | Downloading package conll2007 to /root/nltk_data...
[nltk_data] | Package conll2007 is already up-to-date!
[nltk_data] | Downloading package crubadan to /root/nltk_data...
[nltk_data] | Package crubadan is already up-to-date!
[nltk_data] | Downloading package dependency_treebank to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package dependency_treebank is already up-to-date!
[nltk_data] | Downloading package dolch to /root/nltk_data...
[nltk_data] | Package dolch is already up-to-date!
[nltk_data] | Downloading package europarl_raw to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package europarl_raw is already up-to-date!
[nltk_data] | Downloading package floresta to /root/nltk_data...
[nltk_data] | Package floresta is already up-to-date!
[nltk_data] | Downloading package framenet_v15 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package framenet_v15 is already up-to-date!
[nltk_data] | Downloading package framenet_v17 to
[nltk_data] | /root/nltk_data...
[nltk_data] | Package framenet_v17 is already up-to-date!
[nltk_data] | Downloading package gazetteers to /root/nltk_data...
[nltk_data] | Package gazetteers is already up-to-date!
[nltk_data] | Downloading package genesis to /root/nltk_data...
[nltk_data] | Package genesis is already up-to-date!
[nltk_data] | Downloading package gutenberg to /root/nltk_data...
[nltk_data] | Package gutenberg is already up-to-date!
[nltk_data] | Downloading package ieer to /root/nltk_data...
[nltk_data] | Package ieer is already up-to-date!
[nltk_data] | Downloading package inaugural to /root/nltk_data...
[nltk_data] | Package inaugural is already up-to-date!
[nltk_data] | Downloading package indian to /root/nltk_data...
[nltk_data] | Package indian is already up-to-date!
[nltk_data] | Downloading package jeita to /root/nltk_data...
[nltk_data] | Package jeita is already up-to-date!
[nltk_data] | Downloading package kimmo to /root/nltk_data...
[nltk_data] | Package kimmo is already up-to-date!
[nltk_data] | Downloading package knbc to /root/nltk_data...
[nltk_data] | Package knbc is already up-to-date!
[nltk_data] | Downloading package lin_thesaurus to
[nltk_data] | /root/nltk_data...
[nltk_data] 

In [0]:
# Project 2 Import Datasets
train = spark.read.csv("/FileStore/tables/Train.csv", header="true", inferSchema="true")
test = spark.read.csv("/FileStore/tables/Test.csv", header="true", inferSchema="true")

In [0]:
! pip install nltk
! pip install spark-nlp

Requirement already satisfied: nltk in /databricks/python3/lib/python3.8/site-packages (3.6.5)
Requirement already satisfied: tqdm in /databricks/python3/lib/python3.8/site-packages (from nltk) (4.62.3)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from nltk) (8.0.3)
Requirement already satisfied: regex>=2021.8.3 in /databricks/python3/lib/python3.8/site-packages (from nltk) (2021.11.10)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from nltk) (1.0.1)
 WARNING: You are using pip version 21.0.1; however, version 21.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: spark-nlp in /databricks/python3/lib/python3.8/site-packages (3.3.2)
 WARNING: You are using pip version 21.0.1; however, version 21.3.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [0]:
# user defined functions
def label_convert(r):
    """
    Convert string lables into integer so that it could be processed by machine learning algortihms
    """
    if r == "Depression":
        return 0
    elif r == "Drugs":
        return 1
    elif r == "Alcohol":
        return 2
    elif r == "Suicide":
        return 3
    
def clean_text(r):
    """
    clean text
    """
    nltk.download('stopwords')
    r = r.lower()
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    r_tokens = tokenizer.tokenize(r)
    stop_words = set(stopwords.words('english'))
    filtered_r = [w for w in r_tokens if not w in stop_words]
    return " ".join(filtered_r)

In [0]:
display(train)

ID,text,label
SUAVK39Z,I feel that it was better I dieAm happy,Depression
9JDAGUV3,Why do I get hallucinations?,Drugs
419WR1LQ,I am stresseed due to lack of financial support in school,Depression
6UY7DX6Q,Why is life important?,Suicide
FYC0FTFB,How could I be helped to go through the depression?,Depression
V6VSDJ5I,What are the effects of depression to ones health,Depression
9736J4UE,Why is everything so hard to deal with in this life,Depression
AY8L479Y,I feel emotionally overwhelmed,Depression
OSFJV5EC,How to manage taking alcohol?,Alcohol
U4SGUGGM,Is heaven open for us who smoke bhang?,Drugs


In [0]:
train

Out[197]: DataFrame[ID: string, text: string, label: string, labelnum: int, cleaned_text: string]

In [0]:
display(train.groupby('label').count())

label,count
Drugs,58
Depression,352
Suicide,66
Alcohol,140


In [0]:
# define udf to map label_convert helper function to dataframe
udf_label_convert = F.udf(label_convert, IntegerType())
train= train.withColumn("labelnum", udf_label_convert("label"))
train.show()

+--------+--------------------+----------+--------+
 ID| text| label|labelnum|
+--------+--------------------+----------+--------+
SUAVK39Z|I feel that it wa...|Depression| 0|
9JDAGUV3|Why do I get hall...| Drugs| 1|
419WR1LQ|I am stresseed du...|Depression| 0|
6UY7DX6Q|Why is life impor...| Suicide| 3|
FYC0FTFB|How could I be he...|Depression| 0|
V6VSDJ5I|What are the effe...|Depression| 0|
9736J4UE|Why is everything...|Depression| 0|
AY8L479Y| I feel emotional...|Depression| 0|
OSFJV5EC|How to manage tak...| Alcohol| 2|
U4SGUGGM|Is heaven open fo...| Drugs| 1|
JGBMEUC1|How to survive wi...|Depression| 0|
LM8GPR0X|How do i stop thi...| Alcohol| 2|
EIF8Y8MX|I was ever sad,al...|Depression| 0|
03TVUJ9J|I feel empty,abse...|Depression| 0|
CBMIYRK7|I experienced dep...|Depression| 0|
Y5MH83OX|I feel like am al...|Depression| 0|
OHIWTBLA|I feel low and ou...|Depression| 0|
0LPO6TKU|What should i do ...|Depression| 0|
WQG9NYO0|How can I stop us...| Alcohol| 2|
WYDRQKK9|I feel hopeless, ...|Depression| 0|
+--------+--------------------+----------+--------+
only showing top 20 rows

In [0]:
# define udf to map label_convert helper function to dataframe
udf_clean_text = F.udf(clean_text, StringType())
train = train.withColumn("cleaned_text", udf_clean_text("text"))
train.show()

+--------+--------------------+----------+--------+--------------------+
 ID| text| label|labelnum| cleaned_text|
+--------+--------------------+----------+--------+--------------------+
SUAVK39Z|I feel that it wa...|Depression| 0|feel better dieam...|
9JDAGUV3|Why do I get hall...| Drugs| 1| get hallucinations|
419WR1LQ|I am stresseed du...|Depression| 0|stresseed due lac...|
6UY7DX6Q|Why is life impor...| Suicide| 3| life important|
FYC0FTFB|How could I be he...|Depression| 0|could helped go d...|
V6VSDJ5I|What are the effe...|Depression| 0|effects depressio...|
9736J4UE|Why is everything...|Depression| 0|everything hard d...|
AY8L479Y| I feel emotional...|Depression| 0|feel emotionally ...|
OSFJV5EC|How to manage tak...| Alcohol| 2|manage taking alc...|
U4SGUGGM|Is heaven open fo...| Drugs| 1|heaven open us sm...|
JGBMEUC1|How to survive wi...|Depression| 0|survive without m...|
LM8GPR0X|How do i stop thi...| Alcohol| 2|stop life becomes...|
EIF8Y8MX|I was ever sad,al...|Depression| 0|ever sad alone al...|
03TVUJ9J|I feel empty,abse...|Depression| 0|feel empty absent...|
CBMIYRK7|I experienced dep...|Depression| 0|experienced depre...|
Y5MH83OX|I feel like am al...|Depression| 0|feel like alone w...|
OHIWTBLA|I feel low and ou...|Depression| 0| feel low place|
0LPO6TKU|What should i do ...|Depression| 0| get depression|
WQG9NYO0|How can I stop us...| Alcohol| 2| stop using alcohol|
WYDRQKK9|I feel hopeless, ...|Depression| 0|feel hopeless unw...|
+--------+--------------------+----------+--------+--------------------+
only showing top 20 rows

In [0]:
pd_train = train.toPandas()
pd_train

Out[189]:

,ID,text,label,labelnum,cleaned_text
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression,0,feel better dieam happy
1,9JDAGUV3,Why do I get hallucinations?,Drugs,1,get hallucinations
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression,0,stresseed due lack financial support school
3,6UY7DX6Q,Why is life important?,Suicide,3,life important
4,FYC0FTFB,How could I be helped to go through the depres...,Depression,0,could helped go depression
...,...,...,...,...,...
611,BOHSNXCN,What should I do to stop alcoholism?,Alcohol,2,stop alcoholism
612,GVDXRQPY,How to become my oldself again,Suicide,3,become oldself
613,IO4JHIQS,How can someone stop it?,Alcohol,2,someone stop
614,1DS3P1XO,I feel unworthy,Depression,0,feel unworthy


In [0]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pd_train['cleaned_text'])
Y = pd_train['labelnum']

In [0]:
clf = LogisticRegression(random_state=0).fit(X, Y)
#clf.predict(X)
clf.score(X, Y)

Out[196]: 0.8766233766233766